## LOLA DB Assembly
- LOLA - Locus overlap analysis for enrichment of genomic ranges
- LOLA Documentation
    - http://bioconductor.org/packages/release/bioc/html/LOLA.html
- Encode experiments used to assemble LOLA database are listed in the metadata.tsv file 
    - All Encode V4 release release 1.4.0 - 2.1.2
- Encode bed TFBS files are lifted from hg38 to hg19 if the reference coords are hg38 to align with Illumina 450k coordinates

In [1]:
from collections import Counter
import gzip
import subprocess
import os

import joblib
import pandas as pd

In [2]:
head_dir = os.path.abspath(os.path.join(os.getcwd() ,'..'))

In [3]:
lola_dir = os.path.join(head_dir, 'DataProcessing/loladb')

In [4]:
meta_data = pd.read_csv(f'{head_dir}/DataProcessing/metadata.tsv', sep='\t', index_col=0)

In [5]:
meta_data = meta_data.loc[meta_data['Output type'] == 'IDR thresholded peaks']

In [6]:
Counter([str(x) for x in meta_data['Audit ERROR'].values])

Counter({'nan': 1617})

In [7]:
# the liftOver executable can be downloaded from UCSC here, http://hgdownload.soe.ucsc.edu/admin/exe/
liftover = os.path.join(lola_dir, 'liftOver')

In [8]:
# the chain file can be downloaded from UCSC here, https://hgdownload.soe.ucsc.edu/goldenPath/hg38/liftOver/hg38ToHg19.over.chain.gz

chainfile = os.path.join(lola_dir, 'hg38ToHg19.over.chain.gz')

In [9]:
db_dir = f'{lola_dir}/encode_v4_hg19/'
output_dir = f'{db_dir}regions/'
output_un_dir = f'{lola_dir}/umapped_regions/'
download_dir = f'{lola_dir}/encode_beds/'

In [10]:
for directory in [db_dir, output_dir, output_un_dir, download_dir]:
    if not os.path.exists(directory):
        os.mkdir(directory)

In [11]:
meta_data = meta_data.loc[meta_data['Audit ERROR'].isna()]
meta_data = meta_data.loc[meta_data['Audit NOT_COMPLIANT'].isna()]

In [12]:
def download_encode_bed(download_path, output_directory):
    bed_file = os.path.basename(download_path)
    _bed_exists = os.path.exists(os.path.join(output_directory, bed_file))
    if _bed_exists:
        return 0 
    _wget_command = ['wget', '-q','--limit-rate=5M', '-nd', '-P',
                         output_directory, download_path]
    _w = subprocess.Popen(args=_wget_command)
    _w.wait()    
    if _w.returncode:
        return download_path
    else:
        return 0

In [13]:
download_status = joblib.Parallel(n_jobs=8, verbose=10)(joblib.delayed(download_encode_bed)
                                                       (*[path, download_dir]) for path in meta_data['File download URL'].values)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Batch computation too fast (0.1374s.) Setting batch_size=2.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Batch computation too fast (0.0150s.) Setting batch_size=4.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Batch computation too fast (0.0057s.) Setting batch_size=8.
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 100 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Batch computation too fast (0.0077s.) Setting batch_size=16.
[Parallel(n_jobs=8)]: Done 176 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Batch computation too fast (0.0061s.) Setting batch_size=32.
[Parallel(n_jobs=8)]: Done 320 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Bat

In [14]:
def liftover_bed(lo_binary, chainfile, input_file, output, output_un):
    run = subprocess.run([liftover, '-bedPlus=3', input_file, chainfile, output, output_un])
    return run, ' '.join([liftover, '-bedPlus=3', input_file, chainfile, output, output_un])

In [15]:
lola_index = [['filename', 'sampleType', 'sampleName', 'treatment', 'antibody']]

for sample, sample_info in meta_data.T.to_dict().items():
    hg38 = sample_info['File assembly'] == 'GRCh38'
    sample_type = sample_info['Biosample type']
    sample_type_name = sample_info['Biosample term name']
    treatment = sample_info['Biosample treatments duration']
    target = sample_info['Experiment target']
    antibody = target.replace('-human', '')
    if hg38:
        run = liftover_bed(liftover, chainfile, f'{download_dir}{sample}.bed.gz', 
                     f'{output_dir}{sample}.bed', f'{output_un_dir}{sample}.bed')
        if run[0].returncode:
            print(run)
    else:
        run = subprocess.run(['cp', f'{download_dir}{sample}.bed.gz', f'{output_dir}{sample}.bed.gz'])
        run_1 = subprocess.run(['gunzip', f'{output_dir}{sample}.bed.gz'])
    if run[0].returncode:
            print(sample)
    lola_index.append([f'{sample}.bed', sample_type, sample_type_name, treatment, antibody])

Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping 

In [16]:
with open(f'{db_dir}index.txt', 'w') as out:
    for line in lola_index:
        out.write('\t'.join([str(x) for x in line]) + '\n')